ДЗ 1. Байесовская генерация:

1. написать генератор стилей используя формулу MLE - additive smoothing 

https://stackoverflow.com/questions/37091587/additive-smoothing-for-dataframe-pandas

2. написать генератор для пикселей используя полиномиальную формулу; вероятности считать по каналам пикселей отдельно (из папки avatars)

In [37]:
! ls
print()
! pwd
# название ветки для дз: baes_generation

autoencoder.png  HW_1_Bayes_generator.ipynb       MLE.py     VAE.ipynb
avatars/         images.zip                       Pixels.py
dcgan.ipynb      Intro_generative_modeling.ipynb  vae.gif

/home/iuliiasolomennikova/Desktop/ITMO_ML/GANs


In [38]:
import cv2
import glob
import os
import numpy as np
import pprint
import random
import random
random.seed(10)
np.random.seed(10)

### Генератор стилей используя формулу MLE:
сглаживание : +1 только к нулевым значениям и в конце добавить сколько раз мы прибавляли единицу в знаменатель

найти 0, прибавить 1, и сколько раз встретили - делим


In [39]:
# возможные элементы стиля

styles = {
     'hairstyle':[
         'no hair',
         'long in a bun',
         'long wavy',
         'long straight',
         'short wavy',
         'short straight',
         'short curly'
     ],
     'hair color':[
         'black',
         'blonde',
         'chestnut',
         'pastel pink',
         'ginger',
         'silver grey',
     ],
     'accessory':[
         'no points',
         'round glasses',
         'Sunglasses',
     ],
     'cloth':[
         'hoodie',
         'overalls',
         'T-shirt with round neck',
         'V-neck T-shirt',
     ],
     'clothes color':[
         'black',
         'blue',
         'grey',
         'green',
         'orange',
         'pink',
         'red',
         'white'
     ],
}

styles_count = {
     'hairstyle':[
         7,
         0,
         1,
         23,
         1,
         11,
         7
     ],
     'hair color':[
         7,
         6,
         2,
         3,
         8,
         24,
     ],
     'accessory':[
         11,
         22,
         17,
     ],
     'cloth':[
         7,
         18,
         19,
         6,
     ],
     'clothes color':[
         4,
         5,
         6,
         8,
         6,
         8,
         7,
         6
     ],
}

#1. Probability to meet one element
#2. Find randomly gathered style

In [40]:
styles.keys()

dict_keys(['hairstyle', 'hair color', 'accessory', 'cloth', 'clothes color'])

In [41]:
# count sum of styles:

for st in styles_count.keys():
    summ = 0
    for s in styles_count[st]:
        summ += s
    print(st, summ)

hairstyle 50
hair color 50
accessory 50
cloth 50
clothes color 50


In [42]:
# Define the smoothing parameter
k = 0.5

# Define a function to generate a style
def generate_style(styles, styles_count, k):
    style = {}
    
    for category in styles:
        counts = styles_count[category]
        total_count = sum(counts)
        probabilities = [(count + k) / (total_count + k * len(styles[category])) for count in counts]
        style[category] = random.choices(styles[category], probabilities)[0]
    return style

style = generate_style(styles, styles_count, k)


### Generator itself:

In [45]:
print(style)

{'hairstyle': 'long straight', 'hair color': 'ginger', 'accessory': 'round glasses', 'cloth': 'overalls', 'clothes color': 'red'}


In [46]:
# count sum of styles:
for st in styles_count.keys():
    summ = 0
    for s in styles_count[st]:
        summ += s
    print(st, summ)

proba = {}  # here we place probabilities

for st in styles:

    proba[st] = []
    for num, st_i in enumerate(styles[st]):
        proba_n = (styles_count[st][num] + 1) / ((sum(styles_count[st]) + len(styles_count[st])))
        proba[st].append(proba_n)

print('Counting probabilities:\n')
pprint.pprint(proba)

# random.choices() - возвращает список случайно выбранных элементов из заданного итерабельного списка.

feature = 'accessory'
res_style = {}

for feature in styles:
    prob_acc = proba[feature]
    acc_styles = styles[feature]
    res_style[feature] = random.choices(acc_styles, prob_acc)[0]

if res_style['hairstyle'] == 'no hair':
    res_style.pop('hair color')

print(res_style)
# generate something new from the data
print('------------------------------------------------------')
for i in proba:
    print(i, sum(proba[i]))


hairstyle 50
hair color 50
accessory 50
cloth 50
clothes color 50
Counting probabilities:

{'accessory': [0.22641509433962265, 0.4339622641509434, 0.33962264150943394],
 'cloth': [0.14814814814814814,
           0.35185185185185186,
           0.37037037037037035,
           0.12962962962962962],
 'clothes color': [0.08620689655172414,
                   0.10344827586206896,
                   0.1206896551724138,
                   0.15517241379310345,
                   0.1206896551724138,
                   0.15517241379310345,
                   0.13793103448275862,
                   0.1206896551724138],
 'hair color': [0.14285714285714285,
                0.125,
                0.05357142857142857,
                0.07142857142857142,
                0.16071428571428573,
                0.44642857142857145],
 'hairstyle': [0.14035087719298245,
               0.017543859649122806,
               0.03508771929824561,
               0.42105263157894735,
               0.0350877192982

### Also I created a  style generator pixel by pixel (just for fun) :)


In [34]:

from PIL import Image

# Define the style elements for each pixel channel
red_style = ['red1', 'red2', 'red3', 'red4', 'red5']
green_style = ['green1', 'green2', 'green3', 'green4', 'green5']
blue_style = ['blue1', 'blue2', 'blue3', 'blue4', 'blue5']

# Define the degree for the polimonail
degree = 2

# Define the probabilities of each style element for each pixel channel
red_probabilities = np.array([0.3, 0.2, 0.15, 0.2, 0.15])
green_probabilities = np.array([0.1, 0.15, 0.2, 0.25, 0.3])
blue_probabilities = np.array([0.25, 0.2, 0.15, 0.2, 0.2])

# Use the polynomial to generate weights for each style element for each pixel channel
red_weights = np.polyfit(np.arange(len(red_style)), red_probabilities, degree)
green_weights = np.polyfit(np.arange(len(green_style)), green_probabilities, degree)
blue_weights = np.polyfit(np.arange(len(blue_style)), blue_probabilities, degree)



# function to generate a style pixel by pixel
def generate_pixel_style(red_style, red_weights, green_style, green_weights, blue_style, blue_weights):
    red_weighted_probs = np.polyval(red_weights, np.arange(len(red_style)))
    green_weighted_probs = np.polyval(green_weights, np.arange(len(green_style)))
    blue_weighted_probs = np.polyval(blue_weights, np.arange(len(blue_style)))

    red_idx = np.random.choice(len(red_style), p=red_weighted_probs/red_weighted_probs.sum())
    green_idx = np.random.choice(len(green_style), p=green_weighted_probs/green_weighted_probs.sum())
    blue_idx = np.random.choice(len(blue_style), p=blue_weighted_probs/blue_weighted_probs.sum())

    return (red_style[red_idx], green_style[green_idx], blue_style[blue_idx])



# Generate a style pixel by pixel
width = 10
height = 10
style = np.zeros((height, width, 3), dtype=np.dtype('U10'))


for y in range(height):
    for x in range(width):
        style[y, x] = generate_pixel_style(red_style, red_weights, green_style, green_weights, blue_style, blue_weights)

# generated style
print(style)

# Define the pixel colors for each style element
pixel_colors = {
    'red1': (255, 0, 0),
    'red2': (255, 128, 128),
    'red3': (255, 64, 64),
    'red4': (255, 192, 192),
    'red5': (255, 32, 32),
    'green1': (0, 255, 0),
    'green2': (128, 255, 128),
    'green3': (64, 255, 64),
    'green4': (192, 255, 192),
    'green5': (32, 255, 32),
    'blue1': (0, 0, 255),
    'blue2': (128, 128, 255),
    'blue3': (64, 64, 255),
    'blue4': (192, 192, 255),
    'blue5': (32, 32, 255)
}

# Create an image from the generated style
image = Image.new('RGB', (width, height))

for y in range(height):
    for x in range(width):
        pixel_color = pixel_colors[style[y, x, 0]]
        image.putpixel((x, y), pixel_color)

# Show generated image
image.show()

[[['red4' 'green2' 'blue5']
  ['red3' 'green5' 'blue1']
  ['red1' 'green4' 'blue3']
  ['red4' 'green2' 'blue4']
  ['red4' 'green1' 'blue5']
  ['red5' 'green4' 'blue1']
  ['red2' 'green1' 'blue5']
  ['red1' 'green2' 'blue4']
  ['red1' 'green3' 'blue3']
  ['red5' 'green2' 'blue3']]

 [['red3' 'green1' 'blue2']
  ['red4' 'green4' 'blue5']
  ['red3' 'green2' 'blue3']
  ['red5' 'green4' 'blue4']
  ['red3' 'green5' 'blue2']
  ['red3' 'green3' 'blue1']
  ['red3' 'green5' 'blue3']
  ['red5' 'green2' 'blue5']
  ['red1' 'green2' 'blue1']
  ['red5' 'green5' 'blue3']]

 [['red1' 'green2' 'blue4']
  ['red4' 'green1' 'blue2']
  ['red4' 'green2' 'blue3']
  ['red4' 'green3' 'blue3']
  ['red2' 'green4' 'blue2']
  ['red4' 'green5' 'blue4']
  ['red1' 'green4' 'blue3']
  ['red1' 'green4' 'blue5']
  ['red4' 'green3' 'blue1']
  ['red1' 'green4' 'blue1']]

 [['red1' 'green2' 'blue1']
  ['red1' 'green1' 'blue5']
  ['red2' 'green5' 'blue2']
  ['red1' 'green2' 'blue4']
  ['red3' 'green5' 'blue5']
  ['red3' 'gre

** Message: 17:50:56.615: main.vala:347: Photos Photo Viewer 2.7.4

** (io.elementary.photos:225245): WARNING **: 17:50:57.029: Page.vala:280: Page tmp78eywks0.PNG: Unable to locate action PrevPhoto

** (io.elementary.photos:225245): WARNING **: 17:50:57.029: Page.vala:280: Page tmp78eywks0.PNG: Unable to locate action NextPhoto

** (io.elementary.photos:225245): WARNING **: 17:50:57.029: Page.vala:280: Page tmp78eywks0.PNG: Unable to locate action FlipHorizontally

** (io.elementary.photos:225245): WARNING **: 17:50:57.029: Page.vala:280: Page tmp78eywks0.PNG: Unable to locate action FlipVertically

** (io.elementary.photos:225245): WARNING **: 17:50:57.029: Page.vala:280: Page tmp78eywks0.PNG: Unable to locate action SetBackground

** (io.elementary.photos:225245): WARNING **: 17:50:57.039: DirectoryMonitor.vala:928: Skipping hidden file/directory /tmp/.Test-unix

** (io.elementary.photos:225245): WARNING **: 17:50:57.039: DirectoryMonitor.vala:928: Skipping hidden file/directory /tm